In [ ]:
# default_exp roster

In [ ]:
%load_ext autoreload
%autoreload 2

# Roster

> Module to parse and model a roster with different shifts.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from dataclasses import dataclass
from datetime import datetime, timedelta, date, time
from ics import Calendar, Event
import re
from typing import Optional
from zoneinfo import ZoneInfo


@dataclass
class ShiftProperties:
    name: str
    starting_hour: timedelta
    duration: timedelta


@dataclass
class Shift:
    properties: ShiftProperties
    date: datetime

    def __post_init__(self):
        self.beginning: datetime = self.date + self.properties.starting_hour


In [ ]:
sp = ShiftProperties(
    name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
)
f1 = Shift(properties=sp, date=datetime(2022, 1, 17, tzinfo=ZoneInfo("Europe/Berlin")))
f1.beginning


In [ ]:
# export
@dataclass
class Roster:
    shifts: list[Shift]
    _year: int = 2022
    _month: int = 3 # TODO: Read from Excel
    _dayp = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
    _datep = re.compile(r"\d{2}")

    @classmethod
    def from_dict(
        cls, input: dict[str, str], mapper: Optional[dict] = None
    ) -> "Roster":
        shifts = []
        # TODO: This whole continue stuff is just horrible. Change it future me!
        for date_str, abbr in input.items():
            try:
                props = mapper[abbr]
                if not props:
                    print(f"No properties for shift abbrevation: {abbr}")
                    continue
            except KeyError:
                print(f"Shift abbrevation not found in mapper: {abbr}")
                continue
            date = datetime(
                year=cls._year,
                month=cls._month,
                day=int(cls._datep.search(date_str).group()),
                tzinfo=ZoneInfo("Europe/Berlin"),
            )
            shift = Shift(props, date=date)
            shifts.append(shift)


        return cls(shifts=shifts)
 
    def to_ics(self):
        c = Calendar()
        for shift in self.shifts:
            e = Event()
            e.name = shift.properties.name
            e.begin = shift.beginning
            e.duration = shift.properties.duration
            c.events.add(e)
        return c


In [ ]:
from pathlib import Path

_abbr2shiftproperties = {
    "F1": ShiftProperties(
        name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "N1": ShiftProperties(
        name="Nacht1", starting_hour=timedelta(hours=22), duration=timedelta(hours=10)
    ),
}
r = Roster.from_dict(
    {
        "Mo 01": "F1",
        "Di 02": "N1",
    },
    mapper=_abbr2shiftproperties,
)
c = r.to_ics()
print(c.events)

with (Path.cwd() / "data" / "output" / "example.ics").open("w") as f:
    f.write(str(r.to_ics()))